In [1]:
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_VISIBLE_DEVICES=2


In [4]:
!python ../scripts/chexlocalize/precompute_phrase_embeddings.py \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240207_184445_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)"

Obtaining embeddings for each sentence...
checkpoint_names = ['checkpoint_91_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9337.pt']
  0%|                                                    | 0/10 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=440,3465006188639683439).pkl
len(self.cache["hashes"]) = 4758972
self.cache["embeddings"].shape = (4758972, 128)
100%|███████████████████████████████████████████| 10/10 [00:03<00:00,  2.66it/s]
Saving output to: /mnt/workspace/pamessina/medvqa-workspace/cache/chexlocalize/class_phrase_embeddings(hash=528,1823213522699794899).pkl
Done!


In [5]:
!python ../scripts/chexpert/precompute_phrase_embeddings.py \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240207_184445_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)"

Obtaining embeddings for each sentence...
checkpoint_names = ['checkpoint_91_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9337.pt']
  0%|                                                    | 0/14 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=440,3465006188639683439).pkl
len(self.cache["hashes"]) = 4758972
self.cache["embeddings"].shape = (4758972, 128)
100%|███████████████████████████████████████████| 14/14 [00:03<00:00,  3.90it/s]
Computing embeddings for 1 new texts
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
checkpoint_names = ['checkpoint_91_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sas

In [6]:
from medvqa.utils.files import load_pickle

In [8]:
load_pickle('/mnt/workspace/pamessina/medvqa-workspace/cache/chexpert/class_phrase_embeddings(hash=592,2785101509674370273).pkl')

{'class_phrases': ['no findings',
  'enlarged cardiomediastinum seen',
  'cardiomegaly seen',
  'lung lesion seen',
  'lung opacity seen',
  'edema seen',
  'consolidation seen',
  'pneumonia seen',
  'atelectasis seen',
  'pneumothorax seen',
  'pleural effusion seen',
  'other pleural abnormality seen',
  'fracture seen',
  'support devices seen'],
 'class_phrase_embeddings': array([[ 0.05162085, -0.09709819, -0.02840645, ...,  0.18490952,
          0.01848066, -0.01654728],
        [ 0.01389532,  0.11835788,  0.09294452, ..., -0.11801905,
         -0.04067105, -0.02414907],
        [ 0.01853514,  0.1721783 ,  0.03216044, ..., -0.11712806,
         -0.06561796, -0.01271486],
        ...,
        [-0.03409277,  0.0598366 , -0.01610715, ..., -0.04543193,
          0.08298029,  0.10445303],
        [ 0.11264187, -0.0345632 ,  0.22043173, ..., -0.08498057,
          0.07054407, -0.09380207],
        [ 0.00825152, -0.11535451, -0.19093055, ...,  0.05474152,
          0.07732575, -0.122277

In [21]:
!python ../train_phrase_grounding.py \
--pretrained_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/phrase_grounding/20240505_104907_mim-facts+vinbig_PhraseGrounder(dn121,128,256)/" \
--epochs 200 \
--batches_per_epoch 750 \
--max_images_per_batch 13 \
--max_phrases_per_batch 500 \
--max_phrases_per_image 40 \
--val_batch_size_factor 2.0 \
--raw_image_encoding "densenet-121" \
--image_size 416 416 \
--image_local_feat_size 1024 \
--num_regions 169 \
--regions_width 13 \
--regions_height 13 \
--phrase_embedding_size 128 \
--qkv_size 256 \
--phrase_classifier_hidden_size 256 \
--num_train_workers 2 \
--num_val_workers 2 \
--gradient_accumulation_steps 10 \
--optimizer_name "adamw" \
--scheduler "exp-warmup+decay+cyclicdecay" \
--lr 1e-6 \
--warmup_decay_and_cyclic_decay_args "1e-6,3,1e-4,8,1e-6,1e-4,8,1e-6" \
--use_mimiccxr_facts_for_train \
--use_mimiccxr_facts_for_test \
--dicom_id_to_pos_neg_facts_filepath \
"/mnt/workspace/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_label_based_pos_neg_facts(hash=207,1199289774163830850).pkl" \
--mimiccxr_exclude_noisy_images \
--mimiccxr_balance_long_middle_short_tail \
--mimiccxr_report_fact_nli_integrated_data_filepath \
"/mnt/workspace/pamessina/medvqa-workspace/cache/mimiccxr_integrated_data(hash=1649,2670402002373726730).pkl" \
--binary_multilabel_classif_loss_name "focal+bce+wbce" \
--use_weighted_phrase_classifier_loss \
--cluster_and_label_weights_for_facts_filepath \
"/mnt/workspace/pamessina/medvqa-workspace/cache/mimiccxr_clusters_and_cluster_weights_for_facts(hash=226,607533638975659953).pkl" \
--use_vinbig_for_train \
--vinbig_phrase_embeddings_filepath \
"/mnt/workspace/pamessina/medvqa-workspace/cache/vinbig/label_phrase_embeddings(hash=325,3209166273642513277).pkl" \
--vinbig_training_data_mode "all" \
--use_chexlocalize_for_train \
--chexlocalize_class_phrase_embeddings_filepath \
"/mnt/workspace/pamessina/medvqa-workspace/cache/chexlocalize/class_phrase_embeddings(hash=528,1823213522699794899).pkl" \
--use_chexpert_for_train \
--use_chexpert_for_test \
--chexpert_training_data_mode "all" \
--chexpert_class_phrase_embeddings_filepath \
"/mnt/workspace/pamessina/medvqa-workspace/cache/chexpert/class_phrase_embeddings(hash=592,2785101509674370273).pkl" \
--attention_supervision_loss_weight 3.0 \
--phrase_classifier_loss_weight 1.0 \
--foreground_loss_weight 1.5 \
--mask_exponent 0.4 \
--mimiccxr_facts_weight 5.0 \
--vinbig_weight 1.0 \
--chexpert_weight 3.0 \
--chexlocalize_weight 0.5 \
--save

script's arguments:
   epochs: 200
   batches_per_epoch: 750
   max_images_per_batch: 13
   max_phrases_per_batch: 500
   max_phrases_per_image: 40
   val_batch_size_factor: 2.0
   checkpoint_folder: None
   pretrained_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/phrase_grounding/20240505_104907_mim-facts+vinbig_PhraseGrounder(dn121,128,256)/
   pretrained_checkpoint_folder_paths: None
   freeze_image_encoder: False
   raw_image_encoding: densenet-121
   num_regions: 169
   image_local_feat_size: 1024
   image_encoder_pretrained_weights_path: None
   yolov8_model_name_or_path: None
   yolov8_model_alias: None
   phrase_embedding_size: 128
   regions_width: 13
   regions_height: 13
   qkv_size: 256
   phrase_classifier_hidden_size: 256
   optimizer_name: adamw
   lr: 1e-06
   scheduler: exp-warmup+decay+cyclicdecay
   lr_decay: 0.76
   lr_decay_patience: 2
   warmup_and_decay_args: None
   warmup_and_cosine_args: None
   warmup_decay_and_cyclic_decay_ar

No labels: 3244
Pneumothorax: 25355
Pleural Other: 26329
Lung Lesion: 31041
No Finding: 35666
Pneumonia: 52245
Fracture: 61689
Edema: 94498
Pleural Effusion: 103830
Consolidation: 107292
Cardiomegaly: 122155
Atelectasis: 124175
Support Devices: 136378
Enlarged Cardiomediastinum: 144369
Lung Opacity: 151930
Group sizes: [61396, 30574, 25355, 21621, 18526, 17225, 11524, 11092, 10885, 7081, 4085, 3244, 1170, 347, 173]
  len(indices) = 61396, weight = 4024.122795518331
  len(indices) = 30574, weight = 3307.9607834723624
  len(indices) = 25355, weight = 3131.3487089314644
  len(indices) = 21621, weight = 2986.074594880214
  len(indices) = 18526, weight = 2849.5564265118087
  len(indices) = 17225, weight = 2786.68263628706
  len(indices) = 11524, weight = 2456.196886786416
  len(indices) = 11092, weight = 2426.215857884437
  len(indices) = 10885, weight = 2411.5238383991996
  len(indices) = 7081, weight = 2092.1117706156315
  len(indices) = 4085, weight = 1726.3245386288693
  len(indices) = 

Checkpoint successfully loaded!
--------------------------------------------------
16) Defining log_metrics_handler ...
MetricsLogger :: we'll be logging to /mnt/data/pamessina/workspaces/medvqa-workspace/models/phrase_grounding/20240505_224220_mim-facts+vinbig+chexloc+chxp_PhraseGrounder(dn121,128,256)/metrics_logs.csv
--------------------------------------------------
17) Running trainer engine ...
---- Epoch 1/200
(1) Training stage (lr = 0.000001) ...
loss 1.32487, fg_phrcls_loss 0.90007, vbg_phrcls_loss 0.28210, vbg_prc_auc 0.98627, vbg_att_sup_loss 0.80632, vbg_segmask_iou 0.29265, cl_att_sup_loss 3.34870, cl_segmask_iou 0.18006, cl_phrcls_loss 1.05107, cl_phrase_acc 0.82821, chxp_phrcls_loss 1.60569, 171.17 secs
(2) Validation stage ...
fg_phrcls_loss 1.08414, fg_prc_auc 0.86068, chxp_phrcls_loss 1.01625, chxp_prc_auc 0.66282, 45.91 secs
New checkpoint saved: checkpoint_1_chss+chuc+clss+clcc+clou+fgss+fguc+vbss+vbuc+vbou=0.6121.pt
---- Epoch 2/200
(1) Training stage (lr = 0.0000

loss 1.07363, fg_phrcls_loss 0.91576, vbg_phrcls_loss 0.32829, vbg_prc_auc 0.97526, vbg_att_sup_loss 0.85715, vbg_segmask_iou 0.28256, cl_att_sup_loss 1.10427, cl_segmask_iou 0.39527, cl_phrcls_loss 0.66660, cl_phrase_acc 0.93066, chxp_phrcls_loss 1.18228, 167.63 secs
(2) Validation stage ...
fg_phrcls_loss 1.07608, fg_prc_auc 0.85492, chxp_phrcls_loss 0.71126, chxp_prc_auc 0.88415, 46.03 secs
New checkpoint saved: checkpoint_15_chss+chuc+clss+clcc+clou+fgss+fguc+vbss+vbuc+vbou=0.6884.pt
---- Epoch 16/200
(1) Training stage (lr = 0.000014) ...
loss 1.06573, fg_phrcls_loss 0.91235, vbg_phrcls_loss 0.28713, vbg_prc_auc 0.98361, vbg_att_sup_loss 0.84072, vbg_segmask_iou 0.28201, cl_att_sup_loss 1.11742, cl_segmask_iou 0.39901, cl_phrcls_loss 0.63962, cl_phrase_acc 0.93547, chxp_phrcls_loss 1.18575, 168.11 secs
(2) Validation stage ...
fg_phrcls_loss 1.08920, fg_prc_auc 0.85262, chxp_phrcls_loss 0.71007, chxp_prc_auc 0.89270, 46.00 secs
New checkpoint saved: checkpoint_16_chss+chuc+clss+cl

fg_phrcls_loss 1.07850, fg_prc_auc 0.85421, chxp_phrcls_loss 0.64599, chxp_prc_auc 0.91132, 46.29 secs
---- Epoch 31/200
(1) Training stage (lr = 0.000027) ...
loss 1.01982, fg_phrcls_loss 0.90298, vbg_phrcls_loss 0.29129, vbg_prc_auc 0.98182, vbg_att_sup_loss 0.83112, vbg_segmask_iou 0.28398, cl_att_sup_loss 0.81182, cl_segmask_iou 0.42551, cl_phrcls_loss 0.40603, cl_phrase_acc 0.97315, chxp_phrcls_loss 1.14684, 171.70 secs
(2) Validation stage ...
fg_phrcls_loss 1.09731, fg_prc_auc 0.84822, chxp_phrcls_loss 0.61241, chxp_prc_auc 0.93642, 45.99 secs
New checkpoint saved: checkpoint_31_chss+chuc+clss+clcc+clou+fgss+fguc+vbss+vbuc+vbou=0.7078.pt
---- Epoch 32/200
(1) Training stage (lr = 0.000014) ...
loss 1.02518, fg_phrcls_loss 0.89768, vbg_phrcls_loss 0.26895, vbg_prc_auc 0.98568, vbg_att_sup_loss 0.83438, vbg_segmask_iou 0.29487, cl_att_sup_loss 0.87213, cl_segmask_iou 0.43402, cl_phrcls_loss 0.36934, cl_phrase_acc 0.97578, chxp_phrcls_loss 1.17548, 171.16 secs
(2) Validation stage 

...

fg_phrcls_loss 1.12884, fg_prc_auc 0.85001, chxp_phrcls_loss 0.56945, chxp_prc_auc 0.92958, 46.44 secs
---- Epoch 118/200
(1) Training stage (lr = 0.000052) ...
loss 0.95523, fg_phrcls_loss 0.91135, vbg_phrcls_loss 0.20938, vbg_prc_auc 0.98887, vbg_att_sup_loss 0.74403, vbg_segmask_iou 0.33320, cl_att_sup_loss 0.47293, cl_segmask_iou 0.50243, cl_phrcls_loss 0.08697, cl_phrase_acc 0.99699, chxp_phrcls_loss 1.09326, 170.88 secs
(2) Validation stage ...
fg_phrcls_loss 1.13653, fg_prc_auc 0.83433, chxp_phrcls_loss 0.53134, chxp_prc_auc 0.94579, 46.42 secs
---- Epoch 119/200
(1) Training stage (lr = 0.000027) ...
loss 0.91731, fg_phrcls_loss 0.85120, vbg_phrcls_loss 0.19701, vbg_prc_auc 0.99125, vbg_att_sup_loss 0.74601, vbg_segmask_iou 0.32352, cl_att_sup_loss 0.46188, cl_segmask_iou 0.49761, cl_phrcls_loss 0.07373, cl_phrase_acc 0.99902, chxp_phrcls_loss 1.08404, 170.31 secs
(2) Validation stage ...
fg_phrcls_loss 1.06822, fg_prc_auc 0.85658, chxp_phrcls_loss 0.56845, chxp_prc_auc 0.93428

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



loss 0.90317, fg_phrcls_loss 0.85146, vbg_phrcls_loss 0.16796, vbg_prc_auc 0.99241, vbg_att_sup_loss 0.70340, vbg_segmask_iou 0.33883, cl_att_sup_loss 0.41150, cl_segmask_iou 0.52059, cl_phrcls_loss 0.05160, cl_phrase_acc 0.99923, chxp_phrcls_loss 1.07411, 170.24 secs
(2) Validation stage ...
fg_phrcls_loss 1.05840, fg_prc_auc 0.85640, chxp_phrcls_loss 0.59332, chxp_prc_auc 0.94157, 46.39 secs
---- Epoch 160/200
(1) Training stage (lr = 0.000014) ...
loss 0.90043, fg_phrcls_loss 0.85931, vbg_phrcls_loss 0.14707, vbg_prc_auc 0.99542, vbg_att_sup_loss 0.67771, vbg_segmask_iou 0.33507, cl_att_sup_loss 0.35598, cl_segmask_iou 0.52025, cl_phrcls_loss 0.04035, cl_phrase_acc 0.99960, chxp_phrcls_loss 1.07729, 170.33 secs
(2) Validation stage ...
fg_phrcls_loss 1.10796, fg_prc_auc 0.85023, chxp_phrcls_loss 0.53656, chxp_prc_auc 0.94675, 46.78 secs
---- Epoch 161/200
(1) Training stage (lr = 0.000007) ...
loss 0.89693, fg_phrcls_loss 0.85525, vbg_phrcls_loss 0.16058, vbg_prc_auc 0.99393, vbg_at

fg_phrcls_loss 1.10839, fg_prc_auc 0.85157, chxp_phrcls_loss 0.51506, chxp_prc_auc 0.95407, 46.25 secs
---- Epoch 177/200
(1) Training stage (lr = 0.000007) ...
loss 0.87330, fg_phrcls_loss 0.82356, vbg_phrcls_loss 0.15116, vbg_prc_auc 0.99524, vbg_att_sup_loss 0.68049, vbg_segmask_iou 0.34513, cl_att_sup_loss 0.34880, cl_segmask_iou 0.52840, cl_phrcls_loss 0.03466, cl_phrase_acc 0.99961, chxp_phrcls_loss 1.05068, 170.80 secs
(2) Validation stage ...
fg_phrcls_loss 1.09995, fg_prc_auc 0.85271, chxp_phrcls_loss 0.53000, chxp_prc_auc 0.94942, 45.87 secs
---- Epoch 178/200
(1) Training stage (lr = 0.000004) ...
loss 0.89809, fg_phrcls_loss 0.86142, vbg_phrcls_loss 0.14131, vbg_prc_auc 0.99613, vbg_att_sup_loss 0.67396, vbg_segmask_iou 0.34358, cl_att_sup_loss 0.36270, cl_segmask_iou 0.52201, cl_phrcls_loss 0.03342, cl_phrase_acc 1.00000, chxp_phrcls_loss 1.07119, 169.26 secs
(2) Validation stage ...
fg_phrcls_loss 1.07193, fg_prc_auc 0.85489, chxp_phrcls_loss 0.57351, chxp_prc_auc 0.94224

fg_phrcls_loss 1.09587, fg_prc_auc 0.84979, chxp_phrcls_loss 0.53406, chxp_prc_auc 0.95720, 45.97 secs
---- Epoch 195/200
(1) Training stage (lr = 0.000002) ...
loss 0.87807, fg_phrcls_loss 0.83985, vbg_phrcls_loss 0.14563, vbg_prc_auc 0.99449, vbg_att_sup_loss 0.69199, vbg_segmask_iou 0.34394, cl_att_sup_loss 0.32577, cl_segmask_iou 0.52754, cl_phrcls_loss 0.02535, cl_phrase_acc 1.00000, chxp_phrcls_loss 1.04490, 168.29 secs
(2) Validation stage ...
fg_phrcls_loss 1.10183, fg_prc_auc 0.85212, chxp_phrcls_loss 0.50401, chxp_prc_auc 0.95846, 45.76 secs
---- Epoch 196/200
(1) Training stage (lr = 0.000001) ...
loss 0.87908, fg_phrcls_loss 0.84388, vbg_phrcls_loss 0.14414, vbg_prc_auc 0.99441, vbg_att_sup_loss 0.68804, vbg_segmask_iou 0.34021, cl_att_sup_loss 0.33254, cl_segmask_iou 0.53167, cl_phrcls_loss 0.03031, cl_phrase_acc 1.00000, chxp_phrcls_loss 1.03832, 168.17 secs
(2) Validation stage ...
fg_phrcls_loss 1.07625, fg_prc_auc 0.85413, chxp_phrcls_loss 0.53975, chxp_prc_auc 0.95424